In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
Independent=dataset[['User ID','Gender_Male','Age','EstimatedSalary']]
Independent

,User ID,Gender_Male,Age,EstimatedSalary
0,15624510,1,19,19000
1,15810944,1,35,20000
2,15668575,0,26,43000
3,15603246,0,27,57000
4,15804002,1,19,76000
...,...,...,...,...
395,15691863,0,46,41000
396,15706071,1,51,23000
397,15654296,0,50,20000
398,15755018,1,36,33000


In [7]:
Dependent=dataset[['Purchased']]
Dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'criterion':['gini','entropy','log_loss'],'splitter':['best', 'random'],'max_features':['sqrt','log2']}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [11]:
re=grid.cv_results_
y_pred=grid.predict(x_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[73  6]
 [ 7 34]]


In [12]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.91      0.92      0.92        79
           1       0.85      0.83      0.84        41

    accuracy                           0.89       120
   macro avg       0.88      0.88      0.88       120
weighted avg       0.89      0.89      0.89       120



In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test) 
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("f1_macro value for {}:".format(grid.best_params_),f1_macro)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

f1_macro value for {'criterion': 'log_loss', 'max_features': 'sqrt', 'splitter': 'best'}: 0.89133861324637


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.007426,0.004712,0.014833,0.002891,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.855314,0.785714,0.782971,0.857143,0.909115,0.838051,0.047928,3
1,0.008964,0.006469,0.012678,0.002513,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.742690,0.763089,0.802399,0.718750,0.891667,0.783719,0.060550,10
2,0.006165,0.003618,0.015109,0.006739,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.812540,0.823129,0.785714,0.816856,0.874356,0.822519,0.028892,7
3,0.005616,0.003317,0.016023,0.004824,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.819142,0.876643,0.733523,0.841045,0.838458,0.821762,0.047865,8
4,0.007220,0.002077,0.012490,0.006067,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.808036,0.840114,0.859435,0.727345,0.927778,0.832541,0.065601,5
5,0.003051,0.004067,0.015442,0.004600,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.821429,0.782971,0.726641,0.743599,0.833784,0.781685,0.041895,11
6,0.006136,0.003745,0.009659,0.002552,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.758813,0.874254,0.858503,0.893717,0.909115,0.858880,0.052891,2
7,0.003691,0.004005,0.014179,0.004199,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.758813,0.821429,0.733523,0.800053,0.891667,0.801097,0.054720,9
8,0.004175,0.001828,0.008522,0.002072,log_loss,sqrt,best,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.835985,0.858503,0.859966,0.946663,0.855556,0.871335,0.038644,1
9,0.002440,0.002989,0.009054,0.002474,log_loss,sqrt,random,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.841398,0.700419,0.806153,0.743599,0.690991,0.756512,0.058781,12
